In [1]:
!pip install pymystem3

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pymystem3 import Mystem
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from collections import Counter

In [3]:
df = pd.read_csv('/kaggle/input/testavito/train.csv')
#df = df.sample(10000, random_state = 1234)
#df = df.reset_index()
#df = df.drop('index', axis=1)

In [4]:
#stupid baseline 
for i in range(10):
    df[str(i) +" count"] = df['description'].apply(lambda x: x.count(str(i)))

In [5]:
%%time

m = Mystem()
def lemmatize(text):
    return m.lemmatize(text)
#    return "".join(m.lemmatize(text))


df['description'] = df['description'].apply(lemmatize)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


CPU times: user 9min 52s, sys: 3min 19s, total: 13min 12s
Wall time: 39min 48s


In [6]:
#stupid baseline 2
words = ['один', 'два', 'три', 'четыре', 'пять','шесть',"семь", 'восемь', 'девять', 'ноль']
for i in words:
    df[str(i) +" count"] = df['description'].apply(lambda x: x.count(str(i)))


In [7]:
#Разделение выборки
train, valid_and_test = train_test_split(df, test_size = 0.4, random_state = 1234)
valid, test = train_test_split(valid_and_test, test_size = 0.5, random_state = 1234)
valid_and_test= None 

print("Train 60%", train.shape)
print("Valid 20%", valid.shape)
print('Test 20%', test.shape)

Train 60% (540000, 29)
Valid 20% (180000, 29)
Test 20% (180000, 29)


In [8]:
train_features = train.drop(['title','description','subcategory', 'category', 'price', 'region', 'city', 'datetime_submitted'],axis =1)
test_features = test.drop(['title','description','subcategory', 'category', 'price', 'region', 'city', 'datetime_submitted'],axis =1)
valid_features = valid.drop(['title','description','subcategory', 'category', 'price', 'region', 'city', 'datetime_submitted'],axis =1)

train_target = train['is_bad']
test_target = test['is_bad']
valid_target = valid['is_bad']

In [9]:
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = train_features.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2200      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 8,796
Trainable params: 8,796
Non-trainable params: 0
_________________________________________________________________


In [10]:
my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=5)]
model.fit(
    x=train_features, y=train_target, batch_size=None, epochs=50, verbose=1, callbacks=my_callbacks,
    validation_data=(valid_features, valid_target),steps_per_epoch = len(train_target),validation_steps = len(valid_target))


Epoch 1/50
540000/540000 [==============================] - 655s 1ms/step - loss: 0.0876 - accuracy: 0.9953 - val_loss: 0.0105 - val_accuracy: 0.9983
Epoch 2/50
540000/540000 [==============================] - 668s 1ms/step - loss: 0.1618 - accuracy: 0.9954 - val_loss: 0.0437 - val_accuracy: 0.9985
Epoch 3/50
540000/540000 [==============================] - 651s 1ms/step - loss: 0.1060 - accuracy: 0.9980 - val_loss: 0.0345 - val_accuracy: 0.9994
Epoch 4/50
540000/540000 [==============================] - 645s 1ms/step - loss: 0.0764 - accuracy: 0.9989 - val_loss: 0.0052 - val_accuracy: 0.9993
Epoch 5/50
540000/540000 [==============================] - 654s 1ms/step - loss: 0.0850 - accuracy: 0.9985 - val_loss: 0.0209 - val_accuracy: 0.9985
Epoch 6/50
540000/540000 [==============================] - 650s 1ms/step - loss: 0.0977 - accuracy: 0.9982 - val_loss: 0.0405 - val_accuracy: 0.9988
Epoch 7/50
540000/540000 [==============================] - 657s 1ms/step - loss: 0.1186 - accuracy:

тут судя по всему было бы достаточно простой модели:), но пускай будет нейросеть

In [11]:
model.save("model.h5")

In [12]:
pred = model.predict(test_features)

In [13]:
print("roc auc:",roc_auc_score(test_target, pred ))

roc auc: 0.9998306036379114
